---
title: "panel-material-ui Announcement"
date: "2025-05-13"
description: "Announcing the release of panel-material-ui"
author: "Philipp Rudiger"
categories: [announcement, panel]
image: "images/panel-material-ui-components.png"
---

In [1]:
#| echo: false
import hvplot.pandas
import panel as pn
import pandas as pd
import panel_material_ui as pmui

df = pd.read_csv('https://datasets.holoviz.org/penguins/v1/penguins.csv')

pn.extension('tabulator', inline=False)

## Introducing Panel Material UI: A Modern Makeover for Panel Apps

We have been building [Panel](https://panel.holoviz.org) for almost 7 years now. Today, it powers dashboards, AI workflows, and data applications across research and industry, with over **1.2 million downloads per month**. At the same time one piece of feedback we've gotten over and over was that it was **too hard to style Panel components**. So we set out to build a **comprehensive, consistent, and API compatible set of components**, replacing the existing Panel implementations.

Today we’re excited to introduce **[`panel-material-ui`](https://panel-material-ui.holoviz.org)**, a new extension that wraps [Material UI](https://mui.com/) components in Panel. It gives you a modern, cohesive design system out of the box — while staying fully compatible with your existing Panel code.

Try it out today with:

```bash
pip install panel-material-ui
```

or:

```bash
conda install -c conda-forge panel-material-ui
```

In [2]:
#| echo: false
from IPython.display import IFrame

IFrame('https://panel-material-ui.holoviz.org/_static/demo.html', width="100%", height="420px")

## Why?

Panel’s built-in components were added incrementally over time, usually without involving UI/UX experts, leading to inconsistent styles and theming that’s tricky to apply. Customizing the look and feel often meant CSS overrides and one-off workarounds. To remedy the situation we wanted to make sure we maintained compatibility with existing Panel components while exposing as many MUI components as possible.

Why [MUI]((https://mui.com/)) in particular, you might ask? We wanted to build on a library that has:

* **Maturity**: MUI is one of the most widely used React component libraries, with strong community support and robust documentation.
* **Consistent Design**: It implements Google’s Material Design system, providing a polished and consistent user experience out of the box.
* **Customizable and themeable**: MUI has a well-designed theming system that maps cleanly to how we wanted to style Panel apps.
* **Strong accessibility support**: MUI handles many of the hard parts of accessibility for us, giving us a more solid foundation.
* **Component breadth**: It includes a wide range of components — from simple buttons to complex layout and navigation elements — that we can gradually wrap and expose in Panel.

But for our (and your) sanity we also wanted a library that would make it easy to create new components. The tight React implementation just made it a joy to build, particularly with AI assistance. This also means creating custom components building on `panel-material-ui` is a breeze.

## How?

With the addition of [ESM components](https://panel.holoviz.org/how_to/custom_components/index.html#esm-components), that allow you to communicate seamlessly between the Python based Panel object and the model that is rendered on the frontend. Building on the [React integration](https://panel.holoviz.org/reference/custom_components/ReactComponent.html) I quickly discovered that thanks to the simple nature of the problem, LLMs would go a long way. Using a simple prompt, I was able to bootstrap a comprehensive set of components and their respective React implementations using a single prompt. That said, I have always been skeptical of fully auto-generated component wrappers, while nice in theory the UX of a fully automated wrapper library will always suffer, and even the AI generated component set in `panel-material-ui` was no different.

Once the basic structure was in place, we went to work and reviewed, cleaned up and documented each component in detail. At this point I'd like to extend special thanks to [Thuy Do](https://github.com/thuydotm) and [Marc Skov Madsen](https://github.com/MarcSkovMadsen) for their efforts in writing tests, documentation and generally providing feedback.

## What can it do?

Okay, so what can it actually do? Yes, we've added 70+ components, and they're all consistently styled but what else is possible now that wasn't before?

### Theming

Theming is achieved using the `theme_config` parameter, which is present on all Material components. Thanks to theme inheritance the theme can be configured on a parent component and will flow down to all its children.

In [3]:
from panel_material_ui import Card, Button, FloatSlider

Card(
    FloatSlider(label="Child Button", color="primary"),
    Button(label="Child Button", color="primary"),
    title="Parent Card",
    theme_config={
        "palette": {
            "primary": {"main": "#d219c9"},
        },
        "typography": {
            "fontFamily": "monospace"
        }
    },
    margin=10
).servable()

Card(_names=['', ''], design=<class 'panel_material_ui...., margin=10, objects=[FloatSlider(color='primar...], theme_config={'palette': {'primary': {'...}, title='Parent Card')

### Dark Mode

In Panel theming happened on the backend, i.e. if you toggled between the default (light) and the dark theme the application would have to re-run in order to ensure the updated theming would be propagated. In `panel-material-ui` theming happens client-side. By using the `ThemeToggle` or `Page` components in your app you enter managed theming mode where the theme is controlled globally and styles are updated dynamically.

In [4]:
toggle = pmui.ThemeToggle(styles={'margin-left': 'auto'})
plot = df.hvplot.scatter('bill_length_mm', 'bill_depth_mm', c='species', responsive=True, min_height=300)
table = pn.widgets.Tabulator(df.iloc[:, :5], page_size=5, sizing_mode='stretch_width')

pmui.Paper(toggle, plot, table, styles={"overflow": "hidden"}).preview(height=700)

HTML(str, design=<class 'panel_material_ui...., height=700, sizing_mode='fixed', width=800)

### Menus

We created a new class of `Menu` widgets, including `Breadcrumbs`, `List`, `MenuButton` and `SpeedDial` components, which make it easier to add clean navigation interfaces to Panel.

In [5]:
#| echo: false
breadcrumbs = pmui.Breadcrumbs(items=[
    {'label': 'Documentation', 'icon': 'article'},
    {'label': 'Reference Gallery', 'icon': 'category'},
    {'label': 'Menus', 'icon': 'menu'},
    {'label': 'Breadcrumbs', 'icon': 'grain'},
], active=3)

list_menu = pmui.List(
    items=[
        {
            'label': 'Home',
            'icon': 'home',
            'secondary': 'Overview page',
            'items': [
                {'label': 'Welcome', 'icon': 'handshake'},
                {'label': 'Getting Started', 'icon': 'rocket'}
            ]
        },
        {
            'label': 'Gallery',
            'icon': 'image',
            'secondary': 'Visual overview',
            'selectable': False,
            'items': [
                {'label': 'Charts', 'icon': 'stacked_line_chart'},
                {'label': 'Maps', 'icon': 'map'},
                {'label': 'Animations', 'icon': 'animation'}
            ]
        },
        {
            'label': 'API',
            'icon': 'code',
            'secondary': 'API Reference',
            'items': [
                {'label': 'Endpoints', 'icon': 'terminal'},
                {'label': 'Schemas', 'icon': 'schema'}
            ]
        },
        {
            'label': 'About',
            'icon': 'info',
            'items': [
                {'label': 'Team', 'icon': 'groups'},
                {'label': 'Contact', 'icon': 'mail'}
            ]
        },
    ],
    dense=True,
)

menu_button = pmui.MenuButton(items=[
    {'label': 'Open', 'icon': 'description'},
    {'label': 'Save', 'icon': 'save'},
    {'label': 'Exit', 'icon': 'close'},
], label='File', icon='storage')

speed_dial = pmui.SpeedDial(items=[
    {'label': 'Camera', 'icon': 'camera'},
    {'label': 'Photos', 'icon': 'photo'},
    {'label': 'Documents', 'icon': 'article'},
], active=2, margin=(50, 20))



pmui.Column(
    pmui.Paper(
        '## Breadcrumbs',
        breadcrumbs,
        margin=5,
        sx={'p': '0 10px 10px 10px'}
    ),
    pmui.Row(
    
        pmui.Paper(
            '## List',
            list_menu,
            margin=5,
            sx={'p': '0 10px 10px 10px'}
        ),    
        pmui.Paper(
            '## MenuButton',
            menu_button,
            margin=5,
            sx={'p': '0 10px 10px 10px'}
        ),
        pmui.Paper(
            '## SpeedDial',
            speed_dial,
            margin=5,
            sx={'p': '0 10px 10px 10px'}
        )
    )
)

Column(design=<class 'panel_material_ui...., objects=[Paper(design=<class 'pane...])

### Responsiveness

While Panel could be configured to provide a good mobile experience, it was quite cumbersome. In `panel-material-ui` we are paying attention to the mobile experience, the `Page` automatically adapts to smaller screen by switching the sidebar, the `Grid` component allows configuring the layout per breakpoint and the `BreakpointSwitcher` allows dynamically switching between different components depending on the breakpoints.

Try resizing the output:

In [6]:
from panel_material_ui import Grid

item = lambda content: pmui.Paper(content, elevation=5, height=100, sizing_mode="stretch_width")

small = pmui.Select(options=pmui.COLORS)
large = pmui.RadioButtonGroup(options=pmui.COLORS)

switcher = pmui.BreakpointSwitcher(
    media_query='(min-width: 800px)',
    small=small,
    large=large,
)

grid = Grid(
    Grid(item('Foo'), size={"sm": 8, "xs": 12}),
    Grid(item('Bar'), size={"sm": 4, "xs": 12}),
    Grid(item('Baz'), size={"sm": 4, "xs": 12}),
    Grid(item('Qux'), size={"sm": 8, "xs": 12}),
    container=True, margin=10, spacing=2
)

pmui.Column(switcher, grid).preview(
    styles={'resize': 'horizontal', 'overflow': 'hidden'}, sizing_mode='stretch_width', max_width=850, height=400
)

HTML(str, design=<class 'panel_material_ui...., height=400, max_width=850, sizing_mode='stretch_width', styles={'resize': 'horizontal', ...})

Note how the switcher toggle between large and small implementations and the grid adjusts to the configured breakpoints.

## What's next?

With this being the first public release, our work of course isn't done. Creating, polishing documenting and testing 70+ components is a pretty monumental effort, even with AI assistance. In all likelihood there will be quirks here or there, but we are starting to use the library for our client projects immediately and will fix issues as soon as we uncover them. Thanks to the simplicity of the approach resolving issues will be much simpler than the complex build chain required to develop the legacy components in Bokeh and Panel.

As we incubate these components in the `panel-material-ui` library we are aiming to reach stability and maturity in the coming months and eventually aim to replace the existing components in Panel with this more modern implementation as part of the Panel 2.0 release by the end of the year.

For now we would love for you to try it out, hear your feedback and see what you build with it!